In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [41]:
plt.rc('figure',dpi=100);

In [42]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [43]:
training_labels = training_labels[['person','label']]
#data = data.drop(columns = {'Email','Unknown','Social','days_since_last_session',})
#data = data[['person', 'checkout', 'conversion', 'generic listing', 'search engine hit', 'searched products','staticpage', 'promedio dias', 'visited site','retornos', 'total_time', 'max_session_time','sessions','mean_time_by_session', 'viewed product','max_events_by_session']]
data = data[['person', 'ad campaign hit','Email', 'Organic', 'Paid', 'Social', 'brand listing', 'checkout', 'conversion', 'generic listing', 'search engine hit', 'searched products', 'staticpage', 'viewed product', 'visited site', 'promedio dias', 'retornos', 'longitud busqueda', 'total_time', 'max_session_time', 'mean_events_by_session', 'days_since_last_session']]

In [44]:
data.shape

(38829, 22)

In [45]:
training_labels = training_labels.head(1960)
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [46]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [47]:
RANDOM_SEED = 123
train, test = train_test_split(features,test_size=0.10,random_state=RANDOM_SEED)
print("Train: ",len(train),"Test: ",len(test))
features=list(features.columns)
features.remove('person')
features.remove('label')

x_train=train[features]
y_train=train['label']

x_test=test[features]
y_test=test['label']

Train:  1764 Test:  196


In [48]:
def grid_search():
    n_estimators = [20,30,40,50,60,70,80,90,100]
    min_samples_split = [10,15,20,25,30,35,40,45,50]
    n_jobs = [-1]
    maximos = []
    max_score = 0
    for n_est in n_estimators:
        for min_sam in min_samples_split:
            for n_j in n_jobs:
                c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
                dt=c.fit(x_train,y_train)
                score=c.score(x_test,y_test)*100
                if max_score < score:
                    maximos = [n_est,min_sam,n_j]
                    max_score = score
                #print("n_estimators : "+ str(n_est)+"; min_samples_split: "+str(min_sam)+"; n_job: "+str(n_j)+"; SCORE: "+ str(score))
    print("El mejor score fue: "+str(max_score))
    return maximos

In [ ]:
mejores_hiperparmetros = grid_search()
mejores_hiperparmetros

In [ ]:
c=RandomForestClassifier(n_estimators=50,min_samples_split=40,n_jobs=-1,random_state=0,class_weight="balanced",criterion = 'entropy')
dt=c.fit(x_train,y_train)

In [ ]:
score=c.score(x_test,y_test)*100
score

In [ ]:
prediccion=pd.merge(labels_predict,data,on='person',how='inner')
prediccion = prediccion.fillna(0)

In [ ]:
labels_predict.shape

In [ ]:
x_final=prediccion[features]
y_final=c.predict(x_final)
y_final

In [ ]:
prediccion['label']=y_final
prediccion.head()

In [ ]:
prediccion['label'].value_counts()

In [ ]:
prediccion[['person', 'label']].to_csv('RandomForestResults.csv',index=False)

In [ ]:
# Get numerical feature importances
importances = list(c.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {} Importance: {}'.format(*pair)) for pair in feature_importances];